<a href="https://colab.research.google.com/github/JialinKang/hello-world/blob/master/Question3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Fit a multilayer neural network to the oasis data to predict Gold_Lesions. Use 10-fold cross-validation to compare the performance where you hold out 10% of the observation and fit on the remainder, then repeat this process 10 times. Compare the cross validated accuracy to the cross validated accuracy for logistic regression. Note, you won't be able to fit a very complex NN model on this data because the size is small.**

In [0]:
import pandas as pd
oasis = pd.read_csv('https://raw.githubusercontent.com/bcaffo/ds4bme_intro/master/data/oasis.csv')

In [0]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.utils import np_utils
from keras import layers

In [0]:
oasis_train = oasis[['FLAIR', 'PD', 'T1', 'T2', 'FLAIR_10', 'PD_10', 'T1_10', 'T2_10', 'FLAIR_20', 'PD_20', 'T1_20', 'T2_20']]
oasis_target = oasis[['GOLD_Lesions']]

In [16]:
oasis_train

,FLAIR,PD,T1,T2,FLAIR_10,PD_10,T1_10,T2_10,FLAIR_20,PD_20,T1_20,T2_20
0,1.143692,1.586219,-0.799859,1.634467,0.437568,0.823800,-0.002059,0.573663,0.279832,0.548341,0.219136,0.298662
1,1.652552,1.766672,-1.250992,0.921230,0.663037,0.880250,-0.422060,0.542597,0.422182,0.549711,0.061573,0.280972
2,1.036099,0.262042,-0.858565,-0.058211,-0.044280,-0.308569,0.014766,-0.256075,-0.136532,-0.350905,0.020673,-0.259914
3,1.037692,0.011104,-1.228796,-0.470222,-0.013971,-0.000498,-0.395575,-0.221900,0.000807,-0.003085,-0.193249,-0.139284
4,1.580589,1.730152,-0.860949,1.245609,0.617957,0.866352,-0.099919,0.384261,0.391133,0.608826,0.071648,0.340601
...,...,...,...,...,...,...,...,...,...,...,...,...
95,3.873210,0.539509,1.075184,0.741517,0.490891,-0.072004,0.490218,-0.202590,0.209201,-0.077513,0.209763,-0.102731
96,1.497552,-0.323060,0.751151,-0.151303,0.091900,-0.482855,0.296390,-0.255905,-0.023249,-0.339573,0.115855,-0.151665
97,1.634460,-0.240130,1.530142,-0.817861,0.195918,0.268417,0.378934,0.050704,0.018602,0.165713,0.246406,0.073374
98,1.484775,0.567067,-0.353472,0.542373,-0.005206,-0.063825,0.136869,-0.120705,0.015719,0.093438,0.010343,-0.006837


In [17]:
oasis_target

,GOLD_Lesions
0,0
1,0
2,0
3,0
4,0
...,...
95,1
96,1
97,1
98,1


In [0]:
mean = oasis_train.mean(axis=0)
oasis_train -= mean
std = oasis_train.std(axis=0)
oasis_train /= std

In [0]:
from keras import models
from keras import layers



def oasis_model():
  # model = models.Sequential()
  # model.add(layers.Dense(64, activation='relu',input_shape=(oasis_train.shape[1],)))
  # model.add(layers.Dense(1))
  # model.compile(optimizer='rmsprop',loss='mse',metrics=['mae'])
  model = Sequential() 
  model.add(Dense(96,activation='relu',input_shape=(12,)))
  model.add(Dropout(0.2))
  model.add(Dense(48,activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(1,activation='sigmoid'))
  model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [24]:
import numpy as np

k = 10
num_val_samples = len(oasis_train)//k
num_epochs = 100
all_scores = []
sc = []

for i in range(k):
  print('processing fold #', i)
  val_data = oasis_train[i*num_val_samples:(i+1)*num_val_samples]
  val_targets = oasis_target[i*num_val_samples:(i+1)*num_val_samples]

  partial_train_data = np.concatenate(
      [oasis_train[:i*num_val_samples],
       oasis_train[(i+1)*num_val_samples:]], axis = 0
  )
  partial_train_targets = np.concatenate(
      [oasis_target[:i*num_val_samples],
       oasis_target[(i+1)*num_val_samples:]], axis = 0
  )

  oasis_model_ep = oasis_model()
  oasis_model_ep.fit(partial_train_data, partial_train_targets,
          epochs=num_epochs, batch_size=1,verbose=0)
  val_mse, val_mae = oasis_model_ep.evaluate(val_data, val_targets, verbose=0)
  all_scores.append(val_mae)
  sc.append(val_mse)

processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3
processing fold # 4
processing fold # 5
processing fold # 6
processing fold # 7
processing fold # 8
processing fold # 9


In [25]:
all_scores

[1.0,
 1.0,
 1.0,
 0.800000011920929,
 0.5,
 0.8999999761581421,
 0.8999999761581421,
 0.8999999761581421,
 0.8999999761581421,
 0.800000011920929]

In [30]:
m = 0
for i in all_scores:
  m = m + i

print('NN model accuacy:',m/10)

NN model accuacy: 0.8699999928474427


In [0]:
from sklearn.linear_model import LogisticRegression
import numpy as np

In [27]:
logreg = LogisticRegression()
k = 10
num_val_samples = len(oasis_train)//k
num_epochs = 100
log_scores = []

for i in range(k):
  print('processing fold #', i)
  val_data = np.array(oasis_train[i*num_val_samples:(i+1)*num_val_samples])
  val_targets = np.array(oasis_target[i*num_val_samples:(i+1)*num_val_samples])

  partial_train_data = np.concatenate(
      [oasis_train[:i*num_val_samples],
       oasis_train[(i+1)*num_val_samples:]], axis = 0
  )
  partial_train_targets = np.concatenate(
      [oasis_target[:i*num_val_samples],
       oasis_target[(i+1)*num_val_samples:]], axis = 0
  )

  logreg = LogisticRegression()
  logreg.fit(partial_train_data, partial_train_targets)
  pred = logreg.predict(val_data)
  acc = 1 - (pred.ravel()!=val_targets.ravel()).sum()/val_targets.size
  log_scores.append(acc)

processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3
processing fold # 4
processing fold # 5
processing fold # 6
processing fold # 7
processing fold # 8
processing fold # 9


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

In [28]:
log_scores

[1.0, 1.0, 0.9, 0.8, 0.7, 0.9, 0.9, 0.9, 0.9, 0.8]

In [31]:
m = 0
for i in log_scores:
  m = m + i

print('LogisticRegression model accuacy:',m/10)

LogisticRegression model accuacy: 0.8800000000000002


From the above result:

We can know that logistic regression is better than the NN model